In [ ]:
import os
import sys

__FILE_DIR_PATH = os.path.dirname(__vsc_ipynb_file__) \
    if __IPYTHON__ \
    else os.path.dirname(__file__)

WORKSPACE_HOME = __FILE_DIR_PATH.replace(os.path.basename(__FILE_DIR_PATH), "")
WORKSPACE_HOME = WORKSPACE_HOME.replace("/trunk", "")
DATASET_DIR_HOME = f"{WORKSPACE_HOME}/data/colley"

print(WORKSPACE_HOME)
sys.path.append(WORKSPACE_HOME)

import pickle

from pandas import DataFrame

from core import *
from ipirec import *
from colley import *
from itemcf import *

In [ ]:
RESOURCE_HOME = f"{WORKSPACE_HOME}/resources/similarities"

In [ ]:
_FOLD_SET_ID = 4
top_n_conditions = [n for n in range(3, 37, 2)]

_TEST_SET_FILES_LIST = [
    str.format(
        "{0}/test_{1}_{2}_list.csv",
        DATASET_DIR_HOME,
        _FOLD_SET_ID,
        DecisionType.to_str(d),
    )
    for d in [
        DecisionType.E_LIKE,
        DecisionType.E_PURCHASE,
    ]
]

dataset = ColleyFilteredDataSet(dataset_dir_path=DATASET_DIR_HOME)
dataset._load_metadata_()
for decision_type in DecisionType:
    dataset.append_decisions(
        file_path=str.format(
            "{0}/train_{1}_{2}_list.csv",
            DATASET_DIR_HOME,
            _FOLD_SET_ID,
            DecisionType.to_str(decision_type),
        ),
        decision_type=decision_type,
    )
dataset.__id_index_mapping__()

In [ ]:
file_path = f"{RESOURCE_HOME}/colley_{_FOLD_SET_ID}_item_pcc.bin"
model: Pearson = None
if os.path.exists(file_path):
    with open(file=file_path, mode="rb") as fin:
        model: Pearson = pickle.load(fin)
        fin.close()
else:
    model = Pearson(dataset=dataset)
    model.analysis()

In [ ]:
estimator = AdjustedWeightedSum(model=model)

In [ ]:
recommender = ScoreBasedRecommender(
    estimator=estimator,
)
recommender.prediction()

In [ ]:
"""
recommender = ELABasedRecommender(
    estimator=estimator,
)
recommender.prediction()
"""

In [ ]:
evaluator = IRMetricsEvaluator(
    recommender=recommender,
    file_path=_TEST_SET_FILES_LIST[0],
)
evaluator.top_n_eval(
    top_n_conditions=top_n_conditions,
)
df: DataFrame = evaluator.evlautions_summary_df()
print(df)

In [ ]:
evaluator = IRMetricsEvaluator(
    recommender=recommender,
    file_path=_TEST_SET_FILES_LIST[1],
)
evaluator.top_n_eval(
    top_n_conditions=top_n_conditions,
)
df: DataFrame = evaluator.evlautions_summary_df()
print(df)

In [ ]:
tags_scores_evaluator = TagsScoreRMSEEvaluator(
    recommender=recommender,
    file_path=_TEST_SET_FILES_LIST[0],
)

tags_scores_evaluator.eval()

In [ ]:
tags_scores_evaluator = TagsScoreRMSEEvaluator(
    recommender=recommender,
    file_path=_TEST_SET_FILES_LIST[1],
)

tags_scores_evaluator.eval()

In [ ]:
from rec_tags_freq import CosineItemsTagsFreqAddPenalty

rec_tags_freq_dist = CosineItemsTagsFreqAddPenalty()
evaluator = IRMetricsEvaluator(
    recommender=recommender,
    file_path=_TEST_SET_FILES_LIST[0],
)
avg_cos_dist = rec_tags_freq_dist.tags_freq_distance(
    test_set=evaluator.TEST_SET_LIST,
    recommender=recommender
)
print(f"L: {avg_cos_dist}")

evaluator = IRMetricsEvaluator(
    recommender=recommender,
    file_path=_TEST_SET_FILES_LIST[1],
)
avg_cos_dist = rec_tags_freq_dist.tags_freq_distance(
    test_set=evaluator.TEST_SET_LIST,
    recommender=recommender
)
print(f"P: {avg_cos_dist}")